# Initial Test

* Aluno: Artur Chiaperini Grover
* Exchange: CEX ([CEX.IO](https://cex.io))

In [ ]:
import pickle
import pandas as pd
import numpy as np
import ccxt
import datetime
import requests
import json
import time
import sqlite3
import sqlalchemy
from IPython.display import clear_output

----
## Function definition

In [ ]:
def create_date_list(start, end):
    '''
    create_date_list(start, end):
    returns a list of dates from start to end with format 'YYYYMMDD' and type string.
    
    parameters:
    start -> Starting date with format 'YYYY-MM-DD' (string).
    end   -> Ending date with format 'YYYY-MM-DD' (string).
    '''
    start = datetime.datetime.strptime(start, '%Y-%m-%d')
    end = datetime.datetime.strptime(end, '%Y-%m-%d')
    step = datetime.timedelta(days = 1)
    date_list = list()
    while start <= end:
        date_list.append(str(start.date()).replace('-', ''))
        start += step
        
    return date_list

In [ ]:
def create_url_list(symbol1, symbol2, start, end):
    '''
    create_url_list(symbol1, symbol2, start, end):
    returns a list of urls from start to end date of the symbol1/symbol2 transaction.
    
    parameters:
    symbol1 -> Cryptocurrency acronym (string).
    symbol2 -> Currency acronym (string).
    start   -> Starting date with format 'YYYY-MM-DD' (string).
    end     -> Ending date with format 'YYYY-MM-DD' (string).
    '''
    
    date_list = create_date_list(start, end)
    url_list = list()
    for date in date_list:
        url = 'http://cex.io/api/ohlcv/hd/{}/{}/{}'.format(date, symbol1, symbol2)
        url_list.append(url)
        
    return url_list

In [ ]:
def get_ohlcv(symbol1, symbol2, start, end, data_rate):
    '''
    get_ohlcv(symbol1, symbol2, start, end, data_rate):
    returns a string with all read fetched data from cex.io for the given pair symbol1/symbol2 between
    the starting and ending dates and with data rate (1m, 1h or 1d).
    
    Function has a one second sleep to avoid exploding rate limit of exchange (cex.io).
    Cex rate limit is 600 requests per 10 minutes (1 request per second).
    
    parameters:
    symbol1   -> Cryptocurrency acronym (string).
    symbol2   -> Currency acronym (string).
    start     -> Starting date with format 'YYYY-MM-DD' (string).
    end       -> Ending date with format 'YYYY-MM-DD' (string).
    data_rate -> Data rate, cex.io gives the following options: 'data1m', 'data1h' or 'data1d'.
    '''
    
    url_list = create_url_list(symbol1, symbol2, start, end)
    # time_sleep: guarantees that no more than 1 request are done per second.
    time_sleep = 1.12
    print('Number of urls = {}'.format(len(url_list)))
    print('Time sleep = {}'.format(time_sleep))
    count_input = 0
    data_ohlcv_list = []
    for url in url_list:
        print('Reading {} -----'.format(url))
        request_url = requests.get(url).json()
        if request_url == None:
            print('No data to fetch... -----')
            pass
        else:
            ohlcv = eval(requests.get(url).json()[data_rate])
            data_ohlcv_list.append(ohlcv)
            print('Fetch input data size {}'.format(len(ohlcv)))
            count_input += len(ohlcv)
            print('Finished -----')
        
        time.sleep(time_sleep)
        
    print('Total number of inputs {}'.format(count_input))
    return str([item for sublist in data_ohlcv_list for item in sublist])

----
## Fetching data from Exchange and Saving Pickle File

### Etherium

In [ ]:
# %%time
# cex_eth_usd = get_ohlcv('ETH', 'USD', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_eth_usd = 'ohlcv_eth_usd.pkl'
# pickle.dump(cex_eth_usd, open(file_ohlcv_eth_usd, 'wb'))

In [ ]:
# %%time
# cex_eth_eur = get_ohlcv('ETH', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_eth_eur = 'ohlcv_eth_eur.pkl'
# pickle.dump(cex_eth_eur, open(file_ohlcv_eth_eur, 'wb'))

### Bitcoin

In [ ]:
# %%time
# cex_btc_usd = get_ohlcv('BTC', 'USD', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_btc_usd = 'ohlcv_btc_usd.pkl'
# pickle.dump(cex_btc_usd, open(file_ohlcv_btc_usd, 'wb'))

In [ ]:
# %%time
# cex_btc_eur = get_ohlcv('BTC', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_btc_eur = 'ohlcv_btc_eur.pkl'
# pickle.dump(cex_btc_eur, open(file_ohlcv_btc_eur, 'wb'))

### ZEC - Zcash

In [ ]:
# %%time
# cex_zec_usd = get_ohlcv('ZEC', 'USD', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_zec_usd = 'ohlcv_zec_usd.pkl'
# pickle.dump(cex_zec_usd, open(file_ohlcv_zec_usd, 'wb'))

In [ ]:
# %%time
# cex_zec_eur = get_ohlcv('ZEC', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
# file_ohlcv_zec_eur = 'ohlcv_zec_eur.pkl'
# pickle.dump(cex_zec_eur, open(file_ohlcv_zec_eur, 'wb'))

----
## Renaming and Correcting Datetime

In [ ]:
loaded_ohlcv_eth_usd = pickle.load(open('ohlcv_eth_usd.pkl', "rb"))

In [ ]:
df = pd.read_json(loaded_ohlcv_eth_usd)
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

In [ ]:
format_date = lambda x: datetime.datetime.fromtimestamp(x) 
df['date'] = pd.to_datetime(df['date'].apply(format_date))

In [ ]:
df.set_index('date', inplace = True)

In [ ]:
df

## TESTS

In [1]:
import create_db
from capturer import Capturer
import pandas as pd
from sqlalchemy import create_engine

In [2]:
zec_usd = Capturer('zec/usd')

In [6]:
zec_usd.get_ohlcv('2018-01-01', '2018-01-02')

Number of urls = 2
Time sleep = 1.1
Reading http://cex.io/api/ohlcv/hd/20180101/ZEC/USD -----
Now upload
Fetch input data size 617
Finished -----
Reading http://cex.io/api/ohlcv/hd/20180102/ZEC/USD -----
Now upload
Fetch input data size 742
Finished -----
Total number of inputs 1359


In [4]:
zec_usd.test

[[1514851200, 588.0059, 588.0059, 588, 588, 0.55],
 [1514851260, 588, 588, 588, 588, 1.65121218],
 [1514851500, 588.0005, 588.0005, 588.0005, 588.0005, 0.2739],
 [1514851620, 590.0057, 602.5699, 590.0057, 602.5699, 1.05],
 [1514851800, 588.0122, 588.0122, 585.0001, 588.0064, 18.50290013],
 [1514851860, 585.4154, 585.4154, 585.4095, 585.4095, 0.26063408],
 [1514851920, 600.9997, 600.9997, 600.9997, 600.9997, 0.02507488],
 [1514852220, 600, 607.9999, 600, 607.9999, 12.24459844],
 [1514852400, 595.4284, 595.4284, 595.4224, 595.4224, 0.1156],
 [1514852520, 607.9886, 607.9886, 607.9886, 607.9886, 0.03281311],
 [1514852760, 607.9871, 607.9871, 607.9871, 607.9871, 0.05],
 [1514852880, 595.8514, 607.5673, 595.4284, 607.5673, 1.08565808],
 [1514853060, 607.5673, 607.5673, 607.5673, 607.5673, 0.53681094],
 [1514853120, 607.5673, 607.5673, 607.5673, 607.5673, 0.14440021],
 [1514853360, 596.2867, 604.9999, 596.2867, 596.2867, 0.64156688],
 [1514853480, 600, 600, 600, 600, 0.31853333],
 [1514853780

In [7]:
conn = create_engine('sqlite:///ZEC_USD.sqlite')

df = pd.read_sql_query('select * from zec_usd limit 5;', conn)
df


,open,high,low,close,volume
0,533.0214,533.0214,533.0214,533.0214,0.465000
1,533.0267,533.0267,532.3023,532.3023,2.339496
2,532.3023,532.3023,521.6211,521.6211,28.860926
3,521.6215,521.6215,521.6215,521.6215,0.119527
4,535.7073,535.7073,535.7073,535.7073,0.186202
